In [1]:
import pandas as pd
social_bot1 = pd.read_csv("social_spambots_1/final.csv",sep=',')
social_bot2 = pd.read_csv("social_spambots_2/final.csv",sep=',')
social_bot3 = pd.read_csv("social_spambots_3/final.csv",sep=',')
fake_follower = pd.read_csv('fake_followers/final.csv',sep=',')
real_users = pd.read_csv("genuine_accounts/final.csv",sep=",")
final_train = pd.DataFrame()
final_train = final_train.append(social_bot1.append(social_bot2))
final_train = final_train.append(fake_follower)
final_train = final_train.append(real_users[0:len(real_users)//2 + 1])
final_train = final_train.fillna(0)
final_train.head()

,id,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,verified,...,avg_tweet_activity(sec),max_profile_activity(sec),max_retweet_activity(sec),max_tweet_activity(sec),min_profile_activity(sec),min_retweet_activity(sec),min_tweet_activity(sec),total_retweets,total_tweets,Bot
0,24858289,davideb66,1299,22,40,1,0,1.0,1.0,0.0,...,71246.232243,57323387.0,49616500.0,57323387.0,0.0,173288.0,0.0,11.0,2507.0,1
1,33212890,ElisaDospina,18665,12561,3442,16358,110,0.0,0.0,0.0,...,14724.501259,185536.0,690926.0,295879.0,0.0,1.0,14.0,821.0,2383.0,1
2,39773427,Vladimir65,22987,600,755,14,6,0.0,0.0,0.0,...,8151.565358,353244.0,4069598.0,353244.0,5.0,12.0,5.0,132.0,3084.0,1
3,57007623,RafielaMorales,7975,398,350,11,2,0.0,0.0,0.0,...,30548.202182,3868116.0,6495250.0,3868116.0,2.0,2.0,3.0,663.0,2568.0,1
4,63258466,FabrizioC_c,20218,413,405,162,8,0.0,0.0,0.0,...,7675.933063,2115828.0,2922888.0,2115828.0,1.0,6.0,1.0,242.0,2959.0,1


In [2]:
import time
from datetime import datetime,date
final_train['timestamp'] = pd.to_datetime(final_train['timestamp'])
days = []
for index,row in final_train.iterrows():
    diff = datetime.today() - row['timestamp']
    days.append(diff.days)
final_train['days_active'] = days
del final_train['timestamp']
lengths = []
digits = []
for index,row in final_train.iterrows():
    count = 0
    length = len(row['screen_name'])
    lengths.append(length)
    for character in row["screen_name"]:
        try:
            int(character)
            count += 1
        except:
            continue
    digits.append(count)
final_train['Screen_name_length'] = lengths
final_train['digits_in_name'] = digits
final_train.head()

,id,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,verified,...,max_tweet_activity(sec),min_profile_activity(sec),min_retweet_activity(sec),min_tweet_activity(sec),total_retweets,total_tweets,Bot,days_active,Screen_name_length,digits_in_name
0,24858289,davideb66,1299,22,40,1,0,1.0,1.0,0.0,...,57323387.0,0.0,173288.0,0.0,11.0,2507.0,1,3372,9,2
1,33212890,ElisaDospina,18665,12561,3442,16358,110,0.0,0.0,0.0,...,295879.0,0.0,1.0,14.0,821.0,2383.0,1,3339,12,0
2,39773427,Vladimir65,22987,600,755,14,6,0.0,0.0,0.0,...,353244.0,5.0,12.0,5.0,132.0,3084.0,1,3314,10,2
3,57007623,RafielaMorales,7975,398,350,11,2,0.0,0.0,0.0,...,3868116.0,2.0,2.0,3.0,663.0,2568.0,1,3252,14,0
4,63258466,FabrizioC_c,20218,413,405,162,8,0.0,0.0,0.0,...,2115828.0,1.0,6.0,1.0,242.0,2959.0,1,3230,11,0


In [3]:
bag_of_words_bot = r'bot|b0t|cannabis|tweet me|mishear|follow me|updates every|gorilla|yes_ofc|forget' \
                           r'expos|kill|clit|bbb|butt|fuck|XXX|sex|truthe|fake|anony|free|virus|funky|RNA|kuck|jargon' \
                           r'nerd|swag|jack|bang|bonsai|chick|prison|paper|pokem|xx|freak|ffd|dunia|clone|genie|bbb' \
                           r'ffd|onlyman|emoji|joke|troll|droop|free|every|wow|cheese|yeah|bio|magic|wizard|face'
lengths = []
words = []
for index,row in final_train.iterrows():
    if row["description"] != 0:
        length = len(row["description"])
        if bag_of_words_bot in row["description"]:
            words.append(1)
        else:
            words.append(0)
    else:
        length = 0
        words.append(0)
    lengths.append(length)
final_train["description_length"] = lengths
final_train["contains_words"] = words
final_train.tail()

,id,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,verified,...,min_retweet_activity(sec),min_tweet_activity(sec),total_retweets,total_tweets,Bot,days_active,Screen_name_length,digits_in_name,description_length,contains_words
537,16459227,anonemoose,2111,113,329,99,3,1.0,0.0,0.0,...,8.0,2.0,174.0,1935.0,0,3544,10,0,0,0
538,1978360040,antmanhall5,453,266,1413,169,0,1.0,0.0,0.0,...,1.0,9.0,217.0,236.0,0,1693,11,1,103,0
539,17436585,aprilritchey,84919,3975,1803,14566,153,0.0,0.0,0.0,...,6.0,2.0,584.0,2608.0,0,3492,12,0,94,0
540,589910604,aquieTsui,2042,123,320,1426,0,0.0,0.0,0.0,...,4.0,2.0,112.0,1898.0,0,2207,9,0,53,0
541,11690822,arhemant,1677,834,264,34,28,0.0,0.0,0.0,...,26.0,1.0,220.0,1450.0,0,3814,8,0,152,0


In [4]:
del final_train['id']
final_train.head()

,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,verified,description,...,min_retweet_activity(sec),min_tweet_activity(sec),total_retweets,total_tweets,Bot,days_active,Screen_name_length,digits_in_name,description_length,contains_words
0,davideb66,1299,22,40,1,0,1.0,1.0,0.0,0,...,173288.0,0.0,11.0,2507.0,1,3372,9,2,0,0
1,ElisaDospina,18665,12561,3442,16358,110,0.0,0.0,0.0,Autrice del libro #unavitatuttacurve dal 9 apr...,...,1.0,14.0,821.0,2383.0,1,3339,12,0,134,0
2,Vladimir65,22987,600,755,14,6,0.0,0.0,0.0,[Live Long and Prosper],...,12.0,5.0,132.0,3084.0,1,3314,10,2,23,0
3,RafielaMorales,7975,398,350,11,2,0.0,0.0,0.0,"Cuasi Odontologa*♥,#Bipolar, #Sarcastica & Som...",...,2.0,3.0,663.0,2568.0,1,3252,14,0,149,0
4,FabrizioC_c,20218,413,405,162,8,0.0,0.0,0.0,"I shall rise from my own death, to avenge hers...",...,6.0,1.0,242.0,2959.0,1,3230,11,0,79,0


In [5]:
final_train.to_csv('train_data.csv',index=False)

In [6]:
final_test = pd.DataFrame()
final_test = final_test.append(social_bot3)
final_test = final_test.append(real_users[len(real_users)//2 + 1:len(real_users)])
final_test = final_test.fillna(0)
final_test.head()

,id,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,verified,...,avg_tweet_activity(sec),max_profile_activity(sec),max_retweet_activity(sec),max_tweet_activity(sec),min_profile_activity(sec),min_retweet_activity(sec),min_tweet_activity(sec),total_retweets,total_tweets,Bot
0,16282004,eckates,11405,819,2000,0,19,1.0,0.0,0.0,...,7016.796452,519836.0,6977161.0,519836.0,0.0,70.0,0.0,7.0,3214.0,1
1,16740486,genebailey,520,219,406,36,2,0.0,0.0,0.0,...,279989.782609,10325599.0,10325599.0,58879548.0,0.0,7.0,0.0,236.0,369.0,1
2,17132768,patrickghoward,4671,38877,25953,6,228,0.0,0.0,0.0,...,32412.116547,5591470.0,12788741.0,5591470.0,50.0,62.0,70.0,15.0,3210.0,1
3,18013384,DoggieCakes,8512,2069,1177,70,43,0.0,0.0,0.0,...,12123.285853,68329.0,5442356.0,68329.0,0.0,21.0,0.0,140.0,3097.0,1
4,21331733,DavidVarrone,245,6656,7469,1,72,0.0,0.0,0.0,...,546269.181538,76416002.0,124547654.0,76416002.0,0.0,28.0,0.0,34.0,326.0,1


In [7]:
final_test['timestamp'] = pd.to_datetime(final_test['timestamp'])
days = []
for index,row in final_test.iterrows():
    diff = datetime.today() - row['timestamp']
    days.append(diff.days)
final_test['days_active'] = days
del final_test['timestamp']
lengths = []
digits = []
for index,row in final_test.iterrows():
    count = 0
    length = len(row['screen_name'])
    lengths.append(length)
    for character in row["screen_name"]:
        try:
            int(character)
            count += 1
        except:
            continue
    digits.append(count)
final_test['Screen_name_length'] = lengths
final_test['digits_in_name'] = digits
final_test

,id,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,verified,...,max_tweet_activity(sec),min_profile_activity(sec),min_retweet_activity(sec),min_tweet_activity(sec),total_retweets,total_tweets,Bot,days_active,Screen_name_length,digits_in_name
0,16282004,eckates,11405,819,2000,0,19,1.0,0.0,0.0,...,519836.0,0.0,70.0,0.0,7.0,3214.0,1,3556,7,0
1,16740486,genebailey,520,219,406,36,2,0.0,0.0,0.0,...,58879548.0,0.0,7.0,0.0,236.0,369.0,1,3525,10,0
2,17132768,patrickghoward,4671,38877,25953,6,228,0.0,0.0,0.0,...,5591470.0,50.0,62.0,70.0,15.0,3210.0,1,3506,14,0
3,18013384,DoggieCakes,8512,2069,1177,70,43,0.0,0.0,0.0,...,68329.0,0.0,21.0,0.0,140.0,3097.0,1,3469,11,0
4,21331733,DavidVarrone,245,6656,7469,1,72,0.0,0.0,0.0,...,76416002.0,0.0,28.0,0.0,34.0,326.0,1,3397,12,0
5,22316443,ShowYouSuccess,7778,506,1002,380,8,0.0,0.0,0.0,...,228859.0,0.0,6.0,0.0,38.0,3202.0,1,3388,14,0
6,23088369,mentalitchsite,18685,32089,18275,2,144,0.0,0.0,0.0,...,31899.0,2.0,5.0,2.0,8.0,3199.0,1,3382,14,0
7,26117307,LaurieMoonPR,5473,396,440,194,17,0.0,0.0,0.0,...,1058487.0,0.0,1.0,0.0,726.0,2493.0,1,3365,12,0
8,26743316,21stsoft,61183,33760,16759,5,75,0.0,0.0,0.0,...,197286.0,33.0,0.0,33.0,0.0,3232.0,1,3363,8,2
9,43133380,StratCommun,3910,1319,809,3,49,0.0,0.0,0.0,...,263683.0,0.0,0.0,0.0,553.0,2678.0,1,3299,11,0


In [8]:
bag_of_words_bot = r'bot|b0t|cannabis|tweet me|mishear|follow me|updates every|gorilla|yes_ofc|forget' \
                           r'expos|kill|clit|bbb|butt|fuck|XXX|sex|truthe|fake|anony|free|virus|funky|RNA|kuck|jargon' \
                           r'nerd|swag|jack|bang|bonsai|chick|prison|paper|pokem|xx|freak|ffd|dunia|clone|genie|bbb' \
                           r'ffd|onlyman|emoji|joke|troll|droop|free|every|wow|cheese|yeah|bio|magic|wizard|face'
lengths = []
words = []
for index,row in final_test.iterrows():
    if row["description"] != 0:
        length = len(row["description"])
        if bag_of_words_bot in row["description"]:
            words.append(1)
        else:
            words.append(0)
    else:
        length = 0
        words.append(0)
    lengths.append(length)
final_test["description_length"] = lengths
final_test["contains_words"] = words
final_test.tail()

,id,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,verified,...,min_retweet_activity(sec),min_tweet_activity(sec),total_retweets,total_tweets,Bot,days_active,Screen_name_length,digits_in_name,description_length,contains_words
1078,578727389,FlavDFuego,22317,491,226,7338,6,0.0,0.0,0.0,...,1.0,6.0,223.0,582.0,0,2219,10,0,47,0
1079,181191283,zacaj_,128929,437,304,607,27,0.0,0.0,0.0,...,1.0,0.0,631.0,2573.0,0,2850,6,0,110,0
1080,612711615,zarryftstars,26443,3218,1943,10545,10,0.0,0.0,0.0,...,0.0,5.0,2078.0,1139.0,0,2182,12,0,125,0
1081,599844379,fiveIads,62597,10273,98,39633,77,0.0,0.0,0.0,...,0.0,0.0,1216.0,1923.0,0,2196,8,0,20,0
1082,205218909,zombiemaster999,315,94,597,36,4,0.0,0.0,0.0,...,3562.0,2.0,22.0,282.0,0,2790,15,3,0,0
